In [ ]:
%run "C:\Repos\AICourse\PersonalFinanceAgent\agents\email_classifier_agent.ipynb"

In [2]:
%run "C:\Repos\AICourse\PersonalFinanceAgent\agents\finance_analyst_agent.ipynb"

Using Azure OpenAI service
Kernel created successfully for the finance analyst agent!
Service settings updated: Temperature set to 0.7
Agent 'FinanceAnalystAgent' created successfully!


In [3]:
import asyncio
import os
import asyncio

from typing import Annotated
from openai import AsyncOpenAI


from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)

from semantic_kernel.functions import KernelFunctionFromPrompt

from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.functions import kernel_function

In [4]:
termination_function = KernelFunctionFromPrompt(
    function_name="termination",
    prompt=f"""
    Determine if the user query is answered.
    
    The process is complete when the Finance Analyst agent identifies the best response for the user query.
    Maximum number of turns in a conversation between the {FINANCE_ANALYST_AGENT_NAME} and {EMAIL_CLASSIFIER_AGENT_NAME} is 10.
    
    If the user query is answered by the {FINANCE_ANALYST_AGENT_NAME}, state "yes" otherwise state "no".
    
    History:
    {{{{$history}}}}
    """,
)

selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""
    Determine which participant takes the next turn in a conversation based on the the most recent participant.
    State only the name of the participant to take the next turn.
    No participant should take more than one turn in a row.
    
    Choose only from these participants:
    - {EMAIL_CLASSIFIER_AGENT_NAME}
    - {FINANCE_ANALYST_AGENT_NAME}
    
    Always follow these rules when selecting the next participant, each conversation should be at least 3 turns:
    - After user input, it is {EMAIL_CLASSIFIER_AGENT_NAME}'s turn to get the user emails and classify them.
    - After {EMAIL_CLASSIFIER_AGENT_NAME} replies with the classified set of emails as finance or non-finance, it is {FINANCE_ANALYST_AGENT_NAME}'s turn.
    - {FINANCE_ANALYST_AGENT_NAME} should provide answer to the user query based on the classified emails. If classified emails need more clarity, {FINANCE_ANALYST_AGENT_NAME} can request help from {EMAIL_CLASSIFIER_AGENT_NAME}.

    History:
    {{{{$history}}}}
    """,
)

In [5]:
chat = AgentGroupChat(
    agents=[email_classifier_agent, finance_analyst_agent],
    termination_strategy=KernelFunctionTerminationStrategy(
        agents=[finance_analyst_agent],
        function=termination_function,
        kernel=create_kernel_with_service("termination"),
        result_parser=lambda result: str(result.value[0]).lower() == "yes",
        history_variable_name="history",
        maximum_iterations=10,
    ),
    selection_strategy=KernelFunctionSelectionStrategy(
        function=selection_function,
        kernel=create_kernel_with_service("selection"),
        result_parser=lambda result: str(
            result.value[0]) if result.value is not None else FINANCE_ANALYST_AGENT_NAME,
        agent_variable_name="agents",
        history_variable_name="history",
    ),
)

Using Azure OpenAI service
Using Azure OpenAI service


In [6]:
input = "Please get me the total amount of money spent under each category."

await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=input))
print(f"# User: '{input}'")

async for content in chat.invoke():
    print(f"# Agent - {content.name or '*'}: '{content.content}'")

# User: 'Please get me the total amount of money spent under each category.'
Reading demo data file.
Demo data file read successfully.
# Agent - EmailClassifierAgent: 'Let's process the emails and classify them to determine the relevant finance-related emails to categorize expenses.

### Classified Emails:
```json
[
  {
    "id": 1,
    "subject": "[IMP]: Your Credit Score has ARRIVED!",
    "sender": "Team BankBazaar <creditreport+ratealert@bankbazaar.com>",
    "date": "03 Oct 2024 12:27:04 +0530 (IST)",
    "body": "",
    "email_classification": "Non-Finance"
  },
  {
    "id": 2,
    "subject": "Transaction on XYZ bank account",
    "sender": "tx@xyz.com",
    "date": "04 Oct 2024 18:02:54 +0530",
    "body": "Transaction on XYZ bank account Rs. 900.00 on Oct 04, 2024 at 18:02:54. Info: ABC SUPERMARKET.",
    "email_classification": "Finance"
  },
  {
    "id": 3,
    "subject": "Feb mutual fund investment data, L&T's large contract, & more - Groww Digest",
    "sender": "Groww Di